In [18]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [250]:
#計測地点のosm_idを指定
start_osm = 702526910
mid_osm = [309968200]
end_osm =  211152186
start_mesh = 50325652441
end_mesh = 50325642424

In [251]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [252]:
getdbname = "kddi_06_new_param"
dbname = "kddi_trkn06_monthly_anbun"
trknName = "trkn_06_grp"
sttime = "2022-04-01 06:00:00"

In [253]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値24.4距離0.0 km
50325652244
存在メッシュ：50325652244値28.079001721041912距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値29.14342849291402距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値25.531689075414878距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値27.07047380068945距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値30.233233732683846距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値26.663378502630902距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値26.21362727916153距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値32.447602826585346距離0.8216238562738055 km
50325652222
存在メッシュ：50325652222値31.33646502450523距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値33.56377198380875距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値30.233233732683846距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値34.68333333333333距離1.0498818307447808 km
50325642442
存在メッシュ：50325642442値

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値110.91666666666667距離0.0 km
50325652244
存在メッシュ：50325652244値119.28237074222875距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値121.70277716189904距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値113.49002124171865距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値116.9890730562409距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値124.1808918319105距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値116.06337661673338距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値115.0406846126369距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値129.21615899356982距離0.8216238562738055 km
50325652222
存在メッシュ：50325652222値126.68953608219479距離0.7081817113930013 km
50325642442
50325642442 is kddi none!
50325652224
存在メッシュ：50325652224値124.1808918319105距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値134.3距離1.0498818307447808 km
50325642442
50325642442 is kddi none!
50325642444
存在メッシュ：503256

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値104.83333333333333距離0.0 km
50325652244
存在メッシュ：50325652244値115.30984228612205距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値118.34095709678542距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値108.0559876372295距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値112.43790783545872距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値121.44434089950111距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値111.2786429429322距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値109.99790890786625距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値127.75008887030327距離0.8216238562738055 km
50325652222
存在メッシュ：50325652222値124.58595739825341距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値130.92854788041922距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値121.44434089950111距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値134.11666666666667距離1.0498818307447808 km
50325642442
存

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値24.4距離0.0 km
50325652244
存在メッシュ：50325652244値28.079001721041912距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値29.14342849291402距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値25.531689075414878距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値27.07047380068945距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値30.233233732683846距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値26.663378502630902距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値26.21362727916153距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値32.447602826585346距離0.8216238562738055 km
50325652222
存在メッシュ：50325652222値31.33646502450523距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値33.56377198380875距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値30.233233732683846距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値34.68333333333333距離1.0498818307447808 km
50325642442
存在メッシュ：50325642442値

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値110.91666666666667距離0.0 km
50325652244
存在メッシュ：50325652244値119.28237074222875距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値121.70277716189904距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値113.49002124171865距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値116.9890730562409距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値124.1808918319105距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値116.06337661673338距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値115.0406846126369距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値129.21615899356982距離0.8216238562738055 km
50325652222
存在メッシュ：50325652222値126.68953608219479距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値131.75422273357702距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値124.1808918319105距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値134.3距離1.0498818307447808 km
50325642442
存在メッシュ：503256424

...

50325652441
存在メッシュ：50325652441値105.83333333333333距離0.0 km
50325652244
50325652244 is kddi none!
50325652242
存在メッシュ：50325652242値116.36574343915538距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値108.34615996756085距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値111.76291049207651距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値118.78557030866051距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値110.85898738293517距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値109.86035014443753距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値123.70240281862009距離0.8216238562738055 km
50325652222
50325652222 is kddi none!
50325642442
存在メッシュ：50325642442値126.1807686944646距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値118.78557030866051距離0.595546785455332 km
50325642424
50325642424 is kddi none!
50325642442
存在メッシュ：50325642442値126.1807686944646距離0.9355796797622887 km
50325642444
存在メッシュ：50325642444値123.70240281862009距離0.8216238562738055 km
33.7890625000:33.7984375000:

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値108.38333333333334距離0.0 km
50325652244
存在メッシュ：50325652244値120.9706471633487距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値124.61247036500514距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値112.25528736607372距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値117.52007594800989距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値128.3411233004251距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値116.12724260246436距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値114.58846653642895距離0.18516314374082166 km
50325642444
50325642444 is kddi none!
50325652222
存在メッシュ：50325652222値132.11571204764542距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値139.73620636059474距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値128.3411233004251距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値143.56666666666666距離1.0498818307447808 km
50325642442
存在メッシュ：50325642442値139.73620636059474

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値45.11666666666667距離0.0 km
50325652244
存在メッシュ：50325652244値51.60411216503555距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値53.48109162661878距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値47.11225453384882距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値49.825703125580965距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値55.402822746342565距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値49.10784302138696距離0.23108070982274048 km
50325652424
50325652424 is kddi none!
50325642444
50325642444 is kddi none!
50325652222
存在メッシュ：50325652222値57.34822897892225距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値61.27579781477676距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値55.402822746342565距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値63.25距離1.0498818307447808 km
50325642442
存在メッシュ：50325642442値61.27579781477676距離0.9355796797622887 km
50325642444
50325642444 is kdd

50325652441
存在メッシュ：50325652441値105.83333333333333距離0.0 km
50325652244
50325652244 is kddi none!
50325652242
存在メッシュ：50325652242値116.36574343915538距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値108.34615996756085距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値111.76291049207651距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値118.78557030866051距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値110.85898738293517距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値109.86035014443753距離0.18516314374082166 km
50325642444
存在メッシュ：50325642444値123.70240281862009距離0.8216238562738055 km
50325652222
50325652222 is kddi none!
50325642442
存在メッシュ：50325642442値126.1807686944646距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値118.78557030866051距離0.595546785455332 km
50325642424
50325642424 is kddi none!
50325642442
存在メッシュ：50325642442値126.1807686944646距離0.9355796797622887 km
50325642444
存在メッシュ：50325642444値123.70240281862009距離0.8216238562738055 km
33.7890625000:33.7984375000:

33.7890625000:33.7984375000:132.785156250:132.785156250
50325652441
存在メッシュ：50325652441値108.38333333333334距離0.0 km
50325652244
存在メッシュ：50325652244値120.9706471633487距離0.3756094387877487 km
50325652242
存在メッシュ：50325652242値124.61247036500514距離0.48428259871775 km
50325652423
存在メッシュ：50325652423値112.25528736607372距離0.11554033695271708 km
50325652422
存在メッシュ：50325652422値117.52007594800989距離0.2726432988159301 km
50325652224
存在メッシュ：50325652224値128.3411233004251距離0.595546785455332 km
50325652421
存在メッシュ：50325652421値116.12724260246436距離0.23108070982274048 km
50325652424
存在メッシュ：50325652424値114.58846653642895距離0.18516314374082166 km
50325642444
50325642444 is kddi none!
50325652222
存在メッシュ：50325652222値132.11571204764542距離0.7081817113930013 km
50325642442
存在メッシュ：50325642442値139.73620636059474距離0.9355796797622887 km
50325652224
存在メッシュ：50325652224値128.3411233004251距離0.595546785455332 km
50325642424
存在メッシュ：50325642424値143.56666666666666距離1.0498818307447808 km
50325642442
存在メッシュ：50325642442値139.73620636059474

In [254]:
connection.close()

In [249]:
#CreateGeojsonFile